In [2]:
import torch
import torch.nn as nn
import numpy as np
from tkinter import Tk, Label, Entry, Button
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# Inverse Kinematics SAC Model
class IK_SACModel(nn.Module):
    def __init__(self):
        super(IK_SACModel, self).__init__()
        self.fc = nn.Linear(3, 6)  

    def forward(self, x):
        return self.fc(x)

# Data plotting
def plot_changes(ax, data):
    ax.clear()
    ax.plot(data, marker='o', linestyle='-')
    ax.set_title('Change over Iterations')
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Value')
    ax.figure.canvas.draw()

# GUI with plotting for Inverse Kinematics
def create_gui(model):
    def on_predict():
        # Fetching end-effector position from entries
        pos = [float(entry.get()) for entry in entries]
        joint_angles = predict(model, pos)
        lbl_result.config(text=f'Predicted Joint Angles: {joint_angles}')
        data = np.random.rand(100)
        plot_changes(ax, data)

    root = Tk()
    root.title("IK SAC Robot Predictor")

    # Entries for end-effector position (x, y, z)
    entries = [Entry(root) for _ in range(3)]
    for i, entry in enumerate(entries):
        entry.pack()
        entry.insert(0, f'pos{i+1}')

    btn_predict = Button(root, text="Predict", command=on_predict)
    btn_predict.pack()

    lbl_result = Label(root, text="Predicted Joint Angles: ")
    lbl_result.pack()

    # Plotting setup
    fig = Figure(figsize=(5, 4), dpi=100)
    ax = fig.add_subplot(111)
    canvas = FigureCanvasTkAgg(fig, master=root)
    canvas.draw()
    canvas.get_tk_widget().pack(side='top', fill='both', expand=1)

    root.mainloop()

def predict(model, pos):
    with torch.no_grad():
        pos_tensor = torch.FloatTensor(pos).unsqueeze(0)  # Convert position to tensor
        joint_angles = model(pos_tensor)
        return joint_angles.numpy().squeeze()

if __name__ == "__main__":
    model = IK_SACModel() 
    create_gui(model)
